In [12]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error,  mean_squared_error
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt,ExponentialSmoothing
import statsmodels.api as sm
import seaborn as sns

In [25]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [13]:
#carga de los datos
TablaBase=pd.read_csv(f'BasePron.csv',sep=';')

In [17]:
TablaBase.sample(10)

,fecha,Variable,valor,linea
fecha2,,,,
2020-10-01,1/10/2020,Total clientes,5.536985e+06,pospago
2021-02-01,1/2/2021,Total clientes,2.796205e+06,hogares
2020-08-01,1/8/2020,Total clientes,5.488478e+06,pospago
2020-08-01,1/8/2020,Total clientes,2.751683e+06,hogares
2020-02-01,1/2/2020,CLV TOTAL,5.281040e+12,hogares
2020-05-01,1/5/2020,Total clientes,2.708198e+06,hogares
2020-08-01,1/8/2020,CLV TOTAL,6.279810e+12,pospago
2020-08-01,1/8/2020,ARPU,4.840953e+04,pospago
2020-05-01,1/5/2020,CLV PROMEDIO,2.032451e+06,hogares


##  prueba inicial

In [15]:
TablaBase.set_index('fecha2',inplace=True)

In [16]:
TablaBase.index=pd.to_datetime(TablaBase.index)

In [18]:
#seleccionar una serie
Serie1=TablaBase[(TablaBase['linea']=='pospago')&(TablaBase['Variable']=='CLV TOTAL')]

In [134]:
Serie1

,fecha,Variable,valor,linea
fecha2,,,,
2020-01-01,1/1/2020,CLV TOTAL,5931670000000.000,pospago
2020-02-01,1/2/2020,CLV TOTAL,5970120000000.000,pospago
2020-03-01,1/3/2020,CLV TOTAL,5958330000000.000,pospago
2020-04-01,1/4/2020,CLV TOTAL,6102420000000.000,pospago
2020-05-01,1/5/2020,CLV TOTAL,6192000000000.000,pospago
2020-06-01,1/6/2020,CLV TOTAL,5818950000000.000,pospago
2020-07-01,1/7/2020,CLV TOTAL,6311290000000.000,pospago
2020-08-01,1/8/2020,CLV TOTAL,6279810000000.000,pospago
2020-09-01,1/9/2020,CLV TOTAL,6319770000000.000,pospago


In [48]:
ind = pd.date_range(Serie1.index.max(), periods = 15, freq ='MS', name='FECHA')[1:]
ind

DatetimeIndex(['2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01',
               '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01',
               '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01',
               '2022-11-01', '2022-12-01'],
              dtype='datetime64[ns]', name='FECHA', freq='MS')

In [120]:
model=auto_arima(Serie1['valor'],trace=False,seasonal=True,stepwise=True)

In [121]:
model=model.fit(Serie1['valor'])

In [49]:
#model=auto_arima(Serie1['valor'], start_p=1, start_q=1,max_p=3, max_q=3, m=12,
#                           start_P=0, seasonal=True,
#                           d=1, D=1, trace=False,
#                           error_action='ignore',  
#                           suppress_warnings=True, 
#                           stepwise=True)

In [122]:
forecast= pd.DataFrame(model.predict(n_periods=14),index=ind)

In [62]:
forecast

,0
FECHA,
2021-11-01,6217217967276.721
2021-12-01,6205883984161.527
2022-01-01,6199011581616.698
2022-02-01,6194844474937.771
2022-03-01,6192317734404.123
2022-04-01,6190785635944.378
2022-05-01,6189856642381.686
2022-06-01,6189293343701.999
2022-07-01,6188951785465.990


###  Prueba2

In [85]:
model2=sm.tsa.statespace.SARIMAX(Serie1['valor'],order=(1, 1, 1),seasonal_order=(1,1,1,4))

C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


In [86]:
model2=model2.fit()

C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


In [87]:
num_start=len(Serie1)

In [113]:
forecast2=model2.get_forecast(14)

In [114]:
forecast2.predicted_mean

2021-11-01   6109345671185.834
2021-12-01   6029919901905.705
2022-01-01   6191575918277.396
2022-02-01   6184508826033.918
2022-03-01   6070886176763.002
2022-04-01   5990143372058.445
2022-05-01   6318325045584.201
2022-06-01   6318364741682.641
2022-07-01   6192195558791.734
2022-08-01   6112756756989.046
2022-09-01   6280445917088.074
2022-10-01   6273644493198.081
2022-11-01   6159562588022.545
2022-12-01   6078868770494.856
Freq: MS, Name: predicted_mean, dtype: float64

In [115]:
forecast2.conf_int(0.5)

,lower valor,upper valor
2021-11-01,5645347904051.465,6573343438320.203
2021-12-01,5343012550618.651,6716827253192.759
2022-01-01,5349715029991.449,7033436806563.344
2022-02-01,5207462412428.059,7161555239639.777
2022-03-01,4698207850271.078,7443564503254.926
2022-04-01,4286266366225.052,7694020377891.839
2022-05-01,4348511875510.988,8288138215657.414
2022-06-01,4110082433350.808,8526647050014.474
2022-07-01,3546035331197.124,8838355786386.344
2022-08-01,3069738207482.668,9155775306495.424


In [88]:
forecast=pd.DataFrame(model2.predict(start=num_start,end=num_start+13,dynamic=True),index=ind)

In [89]:
forecast

,predicted_mean
FECHA,
2021-11-01,6109345671185.834
2021-12-01,6029919901905.705
2022-01-01,6191575918277.396
2022-02-01,6184508826033.918
2022-03-01,6070886176763.002
2022-04-01,5990143372058.445
2022-05-01,6318325045584.201
2022-06-01,6318364741682.641
2022-07-01,6192195558791.734


### prueba 3

In [107]:
model3 = sm.tsa.SARIMAX(Serie1['valor'],seasonal_order=(0,1,0,12))
res = model3.fit()
fcast = res.get_forecast(14)

C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


In [108]:
fcast

In [109]:
fcast.predicted_mean

2021-11-01   6367213041792.613
2021-12-01   6367359188238.121
2022-01-01   6467679405209.714
2022-02-01   6512686888956.766
2022-03-01   6475598683718.896
2022-04-01   6335249867852.168
2022-05-01   6217299986733.039
2022-06-01   6093979998668.065
2022-07-01   5982519999866.280
2022-08-01   5900949999986.575
2022-09-01   6235769999998.652
2022-10-01   6235909999999.864
2022-11-01   6367213041792.600
2022-12-01   6367359188238.120
Freq: MS, Name: predicted_mean, dtype: float64

In [110]:
fcast.conf_int(0.5)

,lower valor,upper valor
2021-11-01,5917176023406.449,6817250060178.777
2021-12-01,5915059866580.141,6819658509896.102
2022-01-01,6015357339113.496,6920001471305.932
2022-02-01,6060364593622.353,6965009184291.179
2022-03-01,6023276386073.964,6927920981363.829
2022-04-01,5882927570183.947,6787572165520.389
2022-05-01,5764977689064.584,6669622284401.494
2022-06-01,5641657700999.608,6546302296336.522
2022-07-01,5530197702197.823,6434842297534.737
2022-08-01,5448627702318.118,6353272297655.032


### Prueba 4

In [173]:
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

In [203]:
model4 = ETSModel(Serie1['valor'],seasonal_periods=6,
                  error="mul",
                  trend="mul",
                  seasonal="mul",
                  damped_trend=False,)
#fit = model4.fit(maxiter=10000)
fit = model4.fit()

C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


In [204]:
#pred = fit.get_prediction(start="2014", end="2020")
#pred = fit.get_prediction(14)
pred = fit.get_prediction(start=22, end=22+13)

In [205]:
pred

In [206]:
pred.predicted_mean

2021-11-01   6211928941948.627
2021-12-01   6040434130481.907
2022-01-01   6204432122763.077
2022-02-01   6185801815444.130
2022-03-01   6268973237964.518
2022-04-01   6280518848917.470
2022-05-01   6270171009219.597
2022-06-01   6097068289414.033
2022-07-01   6262603901038.245
2022-08-01   6243798918892.479
2022-09-01   6327750143569.447
2022-10-01   6339404004352.057
2022-11-01   6328959144939.144
2022-12-01   6154233441299.425
Freq: MS, dtype: float64

In [172]:
pred.summary_frame(alpha=0.05)

,mean,pi_lower,pi_upper
2021-11-01,6227167454635.994,5728535389794.465,6725799519477.523
2021-12-01,6320290417330.921,5773085755645.522,6867495079016.319
2022-01-01,6114666374819.778,5522853814336.747,6706478935302.810
2022-02-01,6192955436615.969,5559661324824.570,6826249548407.367
2022-03-01,6354381404639.892,5682153081345.990,7026609727933.793
2022-04-01,6335262317212.962,5626227362088.828,7044297272337.096
2022-05-01,6296659810355.042,5552629985755.593,7040689634954.491
2022-06-01,6320957942988.763,5543500319339.008,7098415566638.518
2022-07-01,6424381886916.994,5614869380134.938,7233894393699.051
2022-08-01,6409028116600.172,5568676519287.714,7249379713912.630
